# Scrapping second website

**Beautiful Soup** is a Python library for pulling data out of HTML and XML files.

In [1]:
#!pip3 install selenium
from selenium import webdriver
import requests

import pandas as pd
from random import randint
from time import sleep
import os.path

#from google.colab import drive
#drive.mount('/content/drive')

brand_file_exists = os.path.exists('../datasets/ultimatespecs-data/brands.csv')
model1_file_exists = os.path.exists('../datasets/ultimatespecs-data/models-1.csv')
model2_file_exists = os.path.exists('../datasets/ultimatespecs-data/models-2.csv')
modelType_file_exists = os.path.exists('../datasets/ultimatespecs-data/modelTypes.csv')
rawData_file_exists = os.path.exists('../datasets/data.csv')

scrapData = 0 # enable=1 or disable=0 to scrap the data

In [2]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup

Source = 'ultimatespecs.com'
CarDataURL = "https://www.ultimatespecs.com"

req = Request('https://www.ultimatespecs.com/car-specs' , headers={'User-Agent': 'Mozilla/5.0'})

webpage = urlopen(req).read()
page_soup = soup(webpage, "html.parser")

In [3]:
def beautiful_soup(url):
    req = Request(url , headers={'User-Agent': 'Mozilla/5.0'})
    try:
        page = urlopen(req).read()
        return soup(page, "html.parser")   
    except:
        return False
            

- '\A\s+|\s+\Z' -> '' will act like strip() removing all leading and trailing whitespace:
  - \A\s+ - matches 1 or more whitespace symbols at the start of the string
  - | - or
  - \s+\Z - matches 1 or more whitespace symbols at the end of the string
- '\n' -> ' ' will replace any newline with a space.

**Function to verify a valid URL**

In [4]:
from urllib.parse import urlparse

def clean_data(df_text):
    return df_text.replace({ r'\A\s+|\s+\Z': '', '\n' : ' '}, regex=True, inplace=True)

def is_url(url):
    try:
        result = urlparse(url)
        return all([result.scheme, result.netloc])
    except ValueError:
        return False

**Functions to:** 
- bypass captcha
- convert audio to text

**NOTE:** For converting audio to text, a local path needs to be set to make the function run

In [5]:
#!pip3 install undetected_chromedriver
#!pip3 install webdriver_manager
#!pip3 install http_request_randomizer
#!pip3 install http_request_randomizer

import undetected_chromedriver as uc
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from http_request_randomizer.requests.proxy.requestProxy import RequestProxy
import os, sys
import time,requests
from bs4 import BeautifulSoup
from random import randint

delayTime = 2
audioToTextDelay = randint(3,5)
filename = '1.mp3'
byPassUrl = 'https://www.ultimatespecs.com/car-specs/BMW/M3639/F13-6-Series-Coupe'
googleIBMLink = 'https://speech-to-text-demo.ng.bluemix.net/'

def audioToText(mp3Path, driver):
    print("1")
    driver.execute_script('''window.open("","_blank");''')
    driver.switch_to.window(driver.window_handles[1])
    print("2")
    driver.get(googleIBMLink)
    delayTime = randint(3,5)
    # Upload file
    sleep(randint(1,3))
    print("3")
    # Upload file
    sleep(randint(1,3))
    root = driver.find_element(By.ID, 'root').find_elements(By.CLASS_NAME, 'dropzone _container _container_large')
    btn = driver.find_element(By.XPATH, '//*[@id="root"]/div/input')
    btn.send_keys('/Volumes/Important/RHUL/Project/dissertation/code/1.mp3') # this path needs to be changed if running in different computer
    # Audio to text is processing
    time.sleep(delayTime)
    #btn.send_keys(path)
    print("4")
    # Audio to text is processing
    time.sleep(audioToTextDelay)
    print("5")
    text = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[7]/div/div/div').find_elements(By.TAG_NAME, 'span')
    print("5.1")
    result = " ".join( [ each.text for each in text ] )
    print("6")
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    print("7")
    return result

def saveFile(content,filename):
    with open(filename, "wb") as handle:
        for data in content.iter_content():
            handle.write(data)

def bypass_captcha(byPassUrl):
    
    option = webdriver.ChromeOptions()
    option.add_argument('--disable-notifications')
    option.add_argument("--mute-audio")
    #option.add_argument('disable-infobars')
    # option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    option.add_argument("user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 10_3 like Mac OS X) AppleWebKit/602.1.50 (KHTML, like Gecko) CriOS/56.0.2924.75 Mobile/14E5239e Safari/602.1")

    driver = uc.Chrome(options=option)


    driver.get(byPassUrl)
    time.sleep(1)
    print(driver.title)
    
    try:
        time.sleep(2)
        popup = driver.find_element(By.ID, 'qc-cmp2-container')
        print('popup', popup)
        driver.switch_to.frame(popup)
        agreeBtn = driver.find_element(By.CLASS_NAME, 'css-1hy2vtq')
        print('agreeBtn', agreeBtn)
        time.sleep(1)
        agreeBtn.click()
        time.sleep(1)
        driver.switch_to.default_content()
    except Exception as e:
        pass
    
    googleClass = driver.find_elements(By.CLASS_NAME, 'g-recaptcha')[0]
    time.sleep(2)
    outeriframe = googleClass.find_element(By.TAG_NAME, 'iframe')
    time.sleep(1)
    outeriframe.click()
    time.sleep(2)
    allIframesLen = driver.find_elements(By.TAG_NAME, 'iframe')
    time.sleep(1)
    audioBtnFound = False
    audioBtnIndex = -1


    #print(allIframesLen)
    for index in range(len(allIframesLen)):
        driver.switch_to.default_content()
        iframe = driver.find_elements(By.TAG_NAME, 'iframe')[index]
        driver.switch_to.frame(iframe)
        print('Click submit button')
        driver.implicitly_wait(delayTime)
        try:
            audioBtn = driver.find_element(By.ID, 'recaptcha-audio-button') or driver.find_element(By.ID, 'recaptcha-anchor')
            audioBtn.click()
            audioBtnFound = True
            audioBtnIndex = index
            break
        except Exception as e:
            pass

    if audioBtnFound:
        try:
            while True:
                href = driver.find_element(By.ID, 'audio-source').get_attribute('src')
                response = requests.get(href, stream=True)
                saveFile(response,filename)
                response = audioToText(os.getcwd() + '/' + filename, driver)
                #print(response)

                driver.switch_to.default_content()
                iframe = driver.find_elements(By.TAG_NAME, 'iframe')[audioBtnIndex]
                driver.switch_to.frame(iframe)

                inputbtn = driver.find_element(By.ID, 'audio-response')
                inputbtn.send_keys(response)
                inputbtn.send_keys(Keys.ENTER)

                time.sleep(2)
                errorMsg = driver.find_elements(By.CLASS_NAME, 'rc-audiochallenge-error-message')[0]

                if errorMsg.text == "" or errorMsg.value_of_css_property('display') == 'none':
                    #print(driver.title)
                    driver.switch_to.default_content()
                    time.sleep(1)
                    driver.switch_to.default_content()
                    #print(driver.title)
                    inputbtn = driver.find_element(By.CLASS_NAME,"showFilterzButton").click()
                    print("Success")
                    return driver

        except Exception as e:
            print(e)
            print('Caught. Need to change proxy now')
            return False

    else:
        print('Button not found. This should not happen.')
        return False

In [6]:
results = page_soup.find_all("div", class_="table_makes")
print(results)

[<div class="table_makes"><div class="row"><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/a-d-Tramontana">a d Tramontana</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Abadal">Abadal</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Abarth-models">Abarth</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Abbott">Abbott</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Abbott-Detroit">Abbott Detroit</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/ABC">ABC</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/AC-models">AC</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Ace">Ace</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/car-specs/Acme">Acme</a></div><div class="makelinks col-sm-6 col-md-3"><a class="highlight_car_specs" href="/car-specs/Acura-models">Acura</a></div><div class="makelinks col-sm-6 col-md-3"><a href="/c

**1. Fetch all the brands**

In [7]:
if(brand_file_exists == True and scrapData == 0):
    CarsData_brands_df = pd.read_csv ('../datasets/ultimatespecs-data/brands.csv')
else:
    CarsData_brands_df = pd.DataFrame(columns=['make','InnerLink'])

    B = 0
    for l, link in enumerate(results):
        brands = link.find_all("a", href=True)
        for i, brand in enumerate(brands):
            inner_url = brand['href'] if (is_url(brand['href']) == True) else CarDataURL+brand['href']
            print(brand.text.strip(), inner_url)

            CarsData_brands_df.loc[B, ['make']] = brand.text.strip()
            CarsData_brands_df.loc[B, ['InnerLink']] = inner_url
            B = B + 1



In [8]:
display(CarsData_brands_df)

# save brands to CSV file after scrapping
if(scrapData == 1):
    CarsData_brands_df.to_csv('../datasets/ultimatespecs-data/brands.csv', encoding='utf-8', index=False)
    print("Brands saved in CSV!")

,make,InnerLink
0,a d Tramontana,https://www.ultimatespecs.com/car-specs/a-d-Tr...
1,Abadal,https://www.ultimatespecs.com/car-specs/Abadal
2,Abarth,https://www.ultimatespecs.com/car-specs/Abarth...
3,Abbott,https://www.ultimatespecs.com/car-specs/Abbott
4,Abbott Detroit,https://www.ultimatespecs.com/car-specs/Abbott...
...,...,...
255,Zeta,https://www.ultimatespecs.com/car-specs/Zeta
256,Zhonghua,https://www.ultimatespecs.com/car-specs/Zhonghua
257,ZIL,https://www.ultimatespecs.com/car-specs/ZIL
258,Zimmerman,https://www.ultimatespecs.com/car-specs/Zimmerman


In [9]:
if(model1_file_exists == True and scrapData == 0):
    CarsData_models_df = pd.read_csv ('../datasets/ultimatespecs-data/models-1.csv')
else:
    CarsData_models_df = pd.DataFrame(columns=['make','model','InnerLink'])

    M = 0
    for b, CD_model_m in CarsData_brands_df.iterrows():
        #if(CD_model_m['Brand'] == 'BMW'):
        print(CD_model_m['make'], CD_model_m['InnerLink'])
        m_soup = beautiful_soup(CD_model_m['InnerLink'])
        
        if(m_soup != False):
            models = m_soup.find_all("div", class_="home_models_line")
            for i, m_models in enumerate(models):
                #print(m_models)

                m_models = m_models.find_all("a", href=True)
                for m, m_model in enumerate(m_models):
                    #print(m_model)
                    title = m_model.find("h2").text.strip()
                    inner_url = m_model['href'] if (is_url(m_model['href']) == True) else CarDataURL+m_model['href']
                    #print(inner_url)

                    CarsData_models_df.loc[M, ['make']] = CD_model_m['make']
                    CarsData_models_df.loc[M, ['model']] = title
                    CarsData_models_df.loc[M, ['InnerLink']] = inner_url
                    M = M + 1

            sleep(randint(3,7))

In [10]:
#clean_data(CarsData_models_df)
CarsData_models_df = CarsData_models_df[CarsData_models_df.model != '']
display(CarsData_models_df)

# save models to CSV file after scrapping
if(scrapData == 1):
    CarsData_models_df.to_csv('../datasets/ultimatespecs-data/models-1.csv', encoding='utf-8', index=False)

,make,model,InnerLink
0,Abarth,595 2017,https://www.ultimatespecs.com/car-specs/Abarth...
1,Abarth,595C 2017,https://www.ultimatespecs.com/car-specs/Abarth...
2,Abarth,124 Spider,https://www.ultimatespecs.com/car-specs/Abarth...
3,Abarth,Grande Punto,https://www.ultimatespecs.com/car-specs/Abarth...
4,Abarth,Punto Evo,https://www.ultimatespecs.com/car-specs/Abarth...
...,...,...,...
2170,Zastava,Yugo Florida,https://www.ultimatespecs.com/car-specs/Zastav...
2171,Zastava,Yugo,https://www.ultimatespecs.com/car-specs/Zastav...
2172,ZAZ,Tavria,https://www.ultimatespecs.com/car-specs/ZAZ/M1...
2173,ZAZ,968,https://www.ultimatespecs.com/car-specs/ZAZ/M1...


In [11]:
def scrap_modelTypes(mt, CD_model_m, CD_mt_df, j):

    #print(mt)
    if(mt != None):
        rows = mt.find_all('tr')
        #print('rows', rows)
        for i, tr in enumerate(rows):
            tds = tr.find_all('td') 
            #print('tds', tds[0])
            m_model = tds[0].find("a")
            if (m_model != None):
                #print('m_model', m_model, m_model['href'], is_url(m_model['href']))
                #if (is_url(m_model['href']) == True):
                inner_url = CarDataURL+m_model['href']

                print(j,inner_url)

                CD_mt_df.loc[j, ['make']] = CD_model_m['make']
                CD_mt_df.loc[j, ['model']] = CD_model_m['model']
                CD_mt_df.loc[j, ['ModelType']] = m_model.text
                CD_mt_df.loc[j, ['ModelTypeLink']] = inner_url
                j=j+1
            #sleep(randint(3,7))

    else:
        mt = mm_soup.find_all("div", class_="home_models_line")
        for i, models in enumerate(mt):
            model_links = models.select("[href]")
            #print(model_links)
            for i, m_model in enumerate(model_links):
                inner_url = CarDataURL+m_model['href']
                print(j,inner_url)

                CD_mt_df.loc[j, ['make']] = CD_model_m['make']
                CD_mt_df.loc[j, ['model']] = CD_model_m['model']
                CD_mt_df.loc[j, ['ModelType']] = m_model.text
                CD_mt_df.loc[j, ['ModelTypeLink']] = inner_url
                j=j+1
    sleep(randint(3,7))
    return(j)
        #print('rows', rows)

In [12]:

if(modelType_file_exists == True and scrapData == 0):
    CD_mt_df = pd.read_csv ('../datasets/ultimatespecs-data/modelTypes.csv')
else:
    CD_mt_df = pd.DataFrame(columns=['make','model','ModelType','ModelTypeLink'])

    j=0
    for m, CD_model_m in CarsData_models_df.iterrows():
        if(CD_model_m['make'] == 'Acura'):
            brand_name = CD_model_m['make']
            mm_soup = beautiful_soup(CD_model_m['InnerLink'])
            print(CD_model_m['make'], CD_model_m['InnerLink'])
            
            ms_results = mm_soup.find_all("table", class_='content_text')
            #print(len(ms_results))
            #print(ms_results)
            
            if(len(ms_results) == 1):
                driver = bypass_captcha(CD_model_m['InnerLink'])
                mm_soup = beautiful_soup(CD_model_m['InnerLink'])
                ms_results = mm_soup.find("table", class_="table_versions")
                j = scrap_modelTypes(mm_soup, CD_model_m, CD_mt_df, j)
            else:
                try:
                    ms_results = mm_soup.find("table", class_="table_versions")
                    j = scrap_modelTypes(ms_results, CD_model_m, CD_mt_df, j)
                except:
                    pass

            

print('Data Scrapped!')

Data Scrapped!


In [13]:
display(CD_mt_df)

# save models to CSV file after scrapping
if(scrapData == 1):
    CD_mt_df.to_csv('../datasets/ultimatespecs-data/modelTypes.csv', encoding='utf-8', index=False)

,make,model,ModelType,ModelTypeLink,Unnamed: 4
0,Abarth,595 2017,595 1.4 16v T-Jet 145 Auto,https://www.ultimatespecs.com/car-specs/Abarth...,NaN
1,Abarth,595 2017,595 1.4 T-Jet 145,https://www.ultimatespecs.com/car-specs/Abarth...,NaN
2,Abarth,595 2017,595 Competizione 1.4 16v T-Jet 180,https://www.ultimatespecs.com/car-specs/Abarth...,NaN
3,Abarth,595 2017,595 Competizione 1.4 16v T-Jet 180 Auto,https://www.ultimatespecs.com/car-specs/Abarth...,NaN
4,Abarth,595 2017,595 Competizione 1.4 T-Jet 180,https://www.ultimatespecs.com/car-specs/Abarth...,NaN
...,...,...,...,...,...
3013,Renault,21,21 Phase 17 Versions,https://www.ultimatespecs.com/car-specs/Renaul...,ultimatespecs.com
3014,Renault,25,25 II6 Versions,https://www.ultimatespecs.com/car-specs/Renaul...,ultimatespecs.com
3015,Renault,25,258 Versions,https://www.ultimatespecs.com/car-specs/Renaul...,ultimatespecs.com
3016,Renault,Safrane,Safrane 26 Versions,https://www.ultimatespecs.com/car-specs/Renaul...,ultimatespecs.com


In [14]:
def scrapper(ms_soup, CD_model_s, sj):

    print('\n', CD_model_s['ModelTypeLink'])
    for t, table in enumerate(ms_results):
        spec_rows = table.find_all("tr", id=True)
        display(table)
        for i, spec_row in enumerate(spec_rows):
            #print("spec_row", spec_row)
            spec = spec_row.find_all("td", class_=True)
            print(i, ":", len(spec), end='')
            if(len(spec) > 0):
                #print("sj", sj, spec[0], spec[1])

                CD_ms_df.loc[sj, ['make']] = CD_model_s['make']
                CD_ms_df.loc[sj, ['model']] = CD_model_s['model']
                CD_ms_df.loc[sj, ['ModelType']] = CD_model_s['ModelType']

                Year = CD_model_s['ModelType'].split()
                CD_ms_df.loc[sj, ['Year']] = Year[0]

                body_type =  spec[1].text.strip() if (spec[0].text.strip() == 'Body :') else ''
                if(body_type != ''):
                    CD_ms_df.loc[sj, ['body_type']] =  body_type

                load_height_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Height :') else ''
                if(load_height_mm != ''):
                    CD_ms_df.loc[sj, ['load_height_mm']] =  load_height_mm

                number_of_seats =  spec[1].text.strip() if (spec[0].text.strip() == 'Num. of Seats :') else ''
                if(number_of_seats != ''):
                    CD_ms_df.loc[sj, ['number_of_seats']] =  number_of_seats

                length_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Length :') else ''
                if(length_mm != ''):
                    CD_ms_df.loc[sj, ['length_mm']] =  length_mm

                width_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Width :') else ''
                if(width_mm != ''):
                    CD_ms_df.loc[sj, ['width_mm']] =  width_mm

                height_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Height :') else ''
                if(height_mm != ''):
                    CD_ms_df.loc[sj, ['height_mm']] =  height_mm

                wheelbase_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Wheelbase :') else ''
                if(wheelbase_mm != ''):
                    CD_ms_df.loc[sj, ['wheelbase_mm']] =  wheelbase_mm

                front_track_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Front Tyres - Rims dimensions :') else ''
                if(front_track_mm != ''):
                    CD_ms_df.loc[sj, ['front_track_mm']] =  front_track_mm

                rear_track_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Rear Tyres - Rims dimensions :') else ''
                if(rear_track_mm != ''):
                    CD_ms_df.loc[sj, ['rear_track_mm']] =  rear_track_mm

                curb_weight_kg =  spec[1].text.strip() if (spec[0].text.strip() == 'Curb Weight :') else ''
                if(curb_weight_kg != ''):
                    CD_ms_df.loc[sj, ['curb_weight_kg']] =  curb_weight_kg

                wheel_size_r14 =  spec[1].text.strip() if (spec[0].text.strip() == 'Wheelbase :') else ''
                if(wheel_size_r14 != ''):
                    CD_ms_df.loc[sj, ['wheel_size_r14']] =  wheel_size_r14

                ground_clearance_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Ground clearance :') else ''
                if(ground_clearance_mm != ''):
                    CD_ms_df.loc[sj, ['ground_clearance_mm']] =  ground_clearance_mm

                trailer_load_with_brakes_kg =  spec[1].text.strip() if (spec[0].text.strip() == 'Trailer with brakes (Kg):') else ''
                if(trailer_load_with_brakes_kg != ''):
                    CD_ms_df.loc[sj, ['trailer_load_with_brakes_kg']] =  trailer_load_with_brakes_kg

                payload_kg =  spec[1].text.strip() if (spec[0].text.strip() == 'Payload (Kg):') else ''
                if(payload_kg != ''):
                    CD_ms_df.loc[sj, ['payload_kg']] =  payload_kg

                front_track_width_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Front Track width (mm)') else ''
                if(front_track_width_mm != ''):
                    CD_ms_df.loc[sj, ['front_track_width_mm']] =  front_track_width_mm

                clearance_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Ground clearance :') else ''
                if(clearance_mm != ''):
                    CD_ms_df.loc[sj, ['clearance_mm']] =  clearance_mm

                full_weight_kg =  spec[1].text.strip() if (spec[0].text.strip() == 'Curb Weight :') else ''
                if(full_weight_kg != ''):
                    CD_ms_df.loc[sj, ['full_weight_kg']] =  full_weight_kg

                front_rear_axle_load_kg =  spec[1].text.strip() if (spec[0].text.strip() == 'Front Rear axle load (kg):') else ''
                if(front_rear_axle_load_kg != ''):
                    CD_ms_df.loc[sj, ['front_rear_axle_load_kg']] =  front_rear_axle_load_kg

                max_trunk_capacity_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Trunk / Boot capacity :') else ''
                if(max_trunk_capacity_l != ''):
                    CD_ms_df.loc[sj, ['max_trunk_capacity_l']] =  max_trunk_capacity_l

                cargo_compartment_length_width_height_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Trunk / Boot capacity :') else ''
                if(cargo_compartment_length_width_height_mm != ''):
                    CD_ms_df.loc[sj, ['cargo_compartment_length_width_height_mm']] =  cargo_compartment_length_width_height_mm

                cargo_volume_m3 =  spec[1].text.strip() if (spec[0].text.strip() == 'Trunk / Boot capacity :') else ''
                if(cargo_volume_m3 != ''):
                    CD_ms_df.loc[sj, ['cargo_volume_m3']] =  cargo_volume_m3


                minimum_trunk_capacity_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Trunk / Boot capacity :') else ''
                if(minimum_trunk_capacity_l != ''):
                    CD_ms_df.loc[sj, ['minimum_trunk_capacity_l']] =  minimum_trunk_capacity_l


                maximum_torque_n_m =  spec[1].text.strip() if (spec[0].text.strip() == 'Maximum torque :') else ''
                if(maximum_torque_n_m != ''):
                    CD_ms_df.loc[sj, ['maximum_torque_n_m']] =  maximum_torque_n_m


                injection_type =  spec[1].text.strip() if (spec[0].text.strip() == 'Fuel System :') else ''
                if(injection_type != ''):
                    CD_ms_df.loc[sj, ['injection_type']] =  injection_type


                overhead_camshaft =  spec[1].text.strip() if (spec[0].text.strip() == 'Overhead camshaft:') else ''
                if(overhead_camshaft != ''):
                    CD_ms_df.loc[sj, ['overhead_camshaft']] =  overhead_camshaft


                cylinder_layout =  spec[1].text.strip() if (spec[0].text.strip() == 'Cylinder layout:') else ''
                if(cylinder_layout != ''):
                    CD_ms_df.loc[sj, ['cylinder_layout']] =  cylinder_layout


                number_of_cylinders =  spec[1].text.strip() if (spec[0].text.strip() == 'Engine type - Number of cylinders :') else ''
                if(number_of_cylinders != ''):
                    CD_ms_df.loc[sj, ['number_of_cylinders']] =  number_of_cylinders


                compression_ratio =  spec[1].text.strip() if (spec[0].text.strip() == 'Compression Ratio :') else ''
                if(compression_ratio != ''):
                    CD_ms_df.loc[sj, ['compression_ratio']] =  compression_ratio
                
                engine_type =  spec[1].text.strip() if (spec[0].text.strip() == 'Engine Type :') else ''
                if(engine_type != ''):
                    CD_ms_df.loc[sj, ['engine_type']] =  engine_type

                valves_per_cylinder =  spec[1].text.strip() if (spec[0].text.strip() == 'Number of valves :') else ''
                if(valves_per_cylinder != ''):
                    CD_ms_df.loc[sj, ['valves_per_cylinder']] =  valves_per_cylinder

                boost_type =  spec[1].text.strip() if (spec[0].text.strip() == 'Boost type:') else ''
                if(boost_type != ''):
                    CD_ms_df.loc[sj, ['boost_type']] =  boost_type

                cylinder_bore_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Cylinder bore (mm):') else ''
                if(cylinder_bore_mm != ''):
                    CD_ms_df.loc[sj, ['cylinder_bore_mm']] =  cylinder_bore_mm

                stroke_cycle_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Stroke cycle (mm):') else ''
                if(stroke_cycle_mm != ''):
                    CD_ms_df.loc[sj, ['stroke_cycle_mm']] =  stroke_cycle_mm

                engine_placement =  spec[1].text.strip() if (spec[0].text.strip() == 'Engine Position :') else ''
                if(engine_placement != ''):
                    CD_ms_df.loc[sj, ['engine_placement']] =  engine_placement

                cylinder_bore_and_stroke_cycle_mm =  spec[1].text.strip() if (spec[0].text.strip() == 'Bore x Stroke :') else ''
                if(cylinder_bore_and_stroke_cycle_mm != ''):
                    CD_ms_df.loc[sj, ['cylinder_bore_and_stroke_cycle_mm']] =  cylinder_bore_and_stroke_cycle_mm

                turnover_of_maximum_torque_rpm =  spec[1].text.strip() if (spec[0].text.strip() == 'Maximum torque :') else ''
                if(turnover_of_maximum_torque_rpm != ''):
                    CD_ms_df.loc[sj, ['turnover_of_maximum_torque_rpm']] =  turnover_of_maximum_torque_rpm

                
                max_power_kw = spec[1].text.strip() if (spec[0].text.strip() == 'Maximum power - Output - Horsepower :') else ''
                if(max_power_kw != ''):
                    CD_ms_df.loc[sj, ['max_power_kw']] = max_power_kw

                presence_of_intercooler =  spec[1].text.strip() if (spec[0].text.strip() == 'Aspiration :') else ''
                if(presence_of_intercooler != ''):
                    CD_ms_df.loc[sj, ['presence_of_intercooler']] =  presence_of_intercooler

                capacity_cm3 =  spec[1].text.strip() if (spec[0].text.strip() == 'Engine size - Displacement - Engine capacity :') else ''
                if(capacity_cm3 != ''):
                    CD_ms_df.loc[sj, ['capacity_cm3']] =  capacity_cm3

                
                engine_hp =  spec[1].text.strip() if (spec[0].text.strip() == 'Maximum power - Output - Horsepower :') else ''
                if(engine_hp != ''):
                    CD_ms_df.loc[sj, ['engine_hp']] = engine_hp

                engine_hp_rpm =  spec[1].text.strip() if (spec[0].text.strip() == 'Maximum power - Output - Horsepower :') else ''
                if(engine_hp_rpm != ''):
                    CD_ms_df.loc[sj, ['engine_hp_rpm']] =  engine_hp_rpm
                    
                drive_wheels = spec[1].text.strip() if (spec[0].text.strip() == 'Drive wheels - Traction - Drivetrain :	') else ''
                if(drive_wheels != ''):
                    CD_ms_df.loc[sj, ['drive_wheels']] = drive_wheels

                bore_stroke_ratio =  spec[1].text.strip() if (spec[0].text.strip() == 'Bore X Stroke:') else ''
                if(bore_stroke_ratio != ''):
                    CD_ms_df.loc[sj, ['bore_stroke_ratio']] =  bore_stroke_ratio

                number_of_gears =  spec[1].text.strip() if (spec[0].text.strip() == 'Transmission Gearbox - Number of speeds :') else ''
                if(number_of_gears != ''):
                    CD_ms_df.loc[sj, ['number_of_gears']] =  number_of_gears

                turning_circle_m =  spec[1].text.strip() if (spec[0].text.strip() == 'Turning radius :') else ''
                if(turning_circle_m != ''):
                    CD_ms_df.loc[sj, ['turning_circle_m']] =  turning_circle_m

                transmission =  spec[1].text.strip() if (spec[0].text.strip() == 'Transmission Gearbox - Number of speeds :') else ''
                if(transmission != ''):
                    CD_ms_df.loc[sj, ['transmission']] =  transmission

                mixed_fuel_consumption_per_100_km_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Fuel Consumption - Economy - Combined WLTP:') else ''
                if(mixed_fuel_consumption_per_100_km_l != ''):
                    CD_ms_df.loc[sj, ['mixed_fuel_consumption_per_100_km_l']] =  mixed_fuel_consumption_per_100_km_l

                range_km =  spec[1].text.strip() if (spec[0].text.strip() == 'Range (WLTP):') else ''
                if(range_km != ''):
                    CD_ms_df.loc[sj, ['range_km']] =  range_km

                emission_standards =  spec[1].text.strip() if (spec[0].text.strip() == 'CO2 emissions :') else ''
                if(emission_standards != ''):
                    CD_ms_df.loc[sj, ['emission_standards']] =  emission_standards

                fuel_tank_capacity_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Fuel Tank Capacity:') else ''
                if(fuel_tank_capacity_l != ''):
                    CD_ms_df.loc[sj, ['fuel_tank_capacity_l']] =  fuel_tank_capacity_l

                acceleration_0_100_km =  spec[1].text.strip() if (spec[0].text.strip() == 'Acceleration 0 to 100 km/h (0 to 62 mph) :') else ''
                if(acceleration_0_100_km != ''):
                    CD_ms_df.loc[sj, ['acceleration_0_100_km/h_s']] =  acceleration_0_100_km

                
                max_speed_km_per_h =  spec[1].text.strip() if (spec[0].text.strip() == 'Top Speed:') else ''
                if(max_speed_km_per_h != ''):
                    CD_ms_df.loc[sj, ['max_speed_km_per_h']] = max_speed_km_per_h

                city_fuel_per_100km_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Fuel Consumption - Economy - High WLTP:Urban Consumption:') else ''
                if(city_fuel_per_100km_l != ''):
                    CD_ms_df.loc[sj, ['city_fuel_per_100km_l']] =  city_fuel_per_100km_l

                CO2_emissions_g =  spec[1].text.strip() if (spec[0].text.strip() == 'CO2 emissions WLTP :') else ''
                if(CO2_emissions_g != ''):
                    CD_ms_df.loc[sj, ['CO2_emissions_g/km']] =  CO2_emissions_g

                fuel_grade =  spec[1].text.strip() if (spec[0].text.strip() == 'Fuel Grade:') else ''
                if(fuel_grade != ''):
                    CD_ms_df.loc[sj, ['fuel_grade']] =  fuel_grade

                highway_fuel_per_100km_l =  spec[1].text.strip() if (spec[0].text.strip() == 'Combined Consumption:') else ''
                if(highway_fuel_per_100km_l != ''):
                    CD_ms_df.loc[sj, ['highway_fuel_per_100km_l']] =  highway_fuel_per_100km_l

                back_suspension =  spec[1].text.strip() if (spec[0].text.strip() == 'Rear Suspension :') else ''
                if(back_suspension != ''):
                    CD_ms_df.loc[sj, ['back_suspension']] =  back_suspension

                rear_brakes =  spec[1].text.strip() if (spec[0].text.strip() == 'Rear Brakes - Dics dimensions :') else ''
                if(rear_brakes != ''):
                    CD_ms_df.loc[sj, ['rear_brakes']] =  rear_brakes

                front_brakes =  spec[1].text.strip() if (spec[0].text.strip() == 'Front Brakes:') else ''
                if(front_brakes != ''):
                    CD_ms_df.loc[sj, ['front_brakes']] =  front_brakes

                front_suspension =  spec[1].text.strip() if (spec[0].text.strip() == 'Front Brakes - Disc dimensions :') else ''
                if(front_suspension != ''):
                    CD_ms_df.loc[sj, ['front_suspension']] =  front_suspension

                steering_type =  spec[1].text.strip() if (spec[0].text.strip() == 'Steering Type :') else ''
                if(steering_type != ''):
                    CD_ms_df.loc[sj, ['steering_type']] =  steering_type

                car_class =  spec[1].text.strip() if (spec[0].text.strip() == 'Car class :') else ''
                if(car_class != ''):
                    CD_ms_df.loc[sj, ['car_class']] =  car_class

                country_of_origin =  spec[1].text.strip() if (spec[0].text.strip() == 'Country of origin :') else ''
                if(country_of_origin != ''):
                    CD_ms_df.loc[sj, ['country_of_origin']] =  country_of_origin

                number_of_doors =  spec[1].text.strip() if (spec[0].text.strip() == 'Num. of Doors :') else ''
                if(number_of_doors != ''):
                    number_of_doors = number_of_doors.split(',')
                    CD_ms_df.loc[sj, ['number_of_doors']] =  number_of_doors[0]

                safety_assessment =  spec[1].text.strip() if (spec[0].text.strip() == 'Safety Assessment:') else ''
                if(safety_assessment != ''):
                    CD_ms_df.loc[sj, ['safety_assessment']] =  safety_assessment

                rating_name =  spec[1].text.strip() if (spec[0].text.strip() == 'Rating :') else ''
                if(rating_name != ''):
                    CD_ms_df.loc[sj, ['rating_name']] =  rating_name

                battery_capacity_KW_per_h =  spec[1].text.strip() if (spec[0].text.strip() == 'Range (NEDC):') else ''
                if(battery_capacity_KW_per_h != ''):
                    CD_ms_df.loc[sj, ['battery_capacity_KW_per_h']] =  battery_capacity_KW_per_h

                electric_range_km =  spec[1].text.strip() if (spec[0].text.strip() == 'Range (WLTP):') else ''
                if(electric_range_km != ''):
                    CD_ms_df.loc[sj, ['electric_range_km']] =  electric_range_km

                charging_time_h =  spec[1].text.strip() if (spec[0].text.strip() == 'Charging Time :') else ''
                if(charging_time_h != ''):
                    CD_ms_df.loc[sj, ['charging_time_h']] =  charging_time_h


                generation =  spec[1].text.strip() if (spec[0].text.strip() == 'Generation :') else ''
                if(generation != ''):
                    CD_ms_df.loc[sj, ['generation']] =  generation

                year_from = CD_model_s['ModelType'].split()
                CD_ms_df.loc[sj, ['year_from']] = year_from[0]

                year_to = CD_model_s['ModelType'].split()
                CD_ms_df.loc[sj+t, ['year_to']] = year_from[0]

                trim =  spec[1].text.strip() if (spec[0].text.strip() == 'Trim:') else ''
                if(trim != ''):
                    CD_ms_df.loc[sj, ['trim']] =  trim

            #print(CD_ms_df)
        (randint(5,15))
        sj=sj+1

    return CD_ms_df, sj

In [15]:
if(rawData_file_exists == True and scrapData == 0):
    CD_ms_df = pd.read_csv ('../datasets/data.csv')
else:
    all_cols = ['make','model','generation','year_from','year_to','series','trim',
                'body_type','load_height_mm','number_of_seats','length_mm','width_mm','height_mm',
                'wheelbase_mm','front_track_mm','rear_track_mm','curb_weight_kg','wheel_size_r14',
                'ground_clearance_mm','trailer_load_with_brakes_kg','payload_kg','back_track_width_mm',
                'front_track_width_mm','clearance_mm','full_weight_kg','front_rear_axle_load_kg',
                'max_trunk_capacity_l','cargo_compartment_length_width_height_mm',
                'cargo_volume_m3','minimum_trunk_capacity_l','maximum_torque_n_m','injection_type',
                'overhead_camshaft','cylinder_layout','number_of_cylinders','compression_ratio',
                'engine_type','valves_per_cylinder','boost_type','cylinder_bore_mm','stroke_cycle_mm',
                'engine_placement','cylinder_bore_and_stroke_cycle_mm','turnover_of_maximum_torque_rpm',
                'max_power_kw','presence_of_intercooler','capacity_cm3','engine_hp','engine_hp_rpm',
                'drive_wheels','bore_stroke_ratio','number_of_gears','turning_circle_m','transmission',
                'mixed_fuel_consumption_per_100_km_l','range_km','emission_standards',
                'fuel_tank_capacity_l','acceleration_0_100_km/h_s','max_speed_km_per_h',
                'city_fuel_per_100km_l','CO2_emissions_g/km','fuel_grade','highway_fuel_per_100km_l',
                'back_suspension','rear_brakes','front_brakes','front_suspension','steering_type',
                'car_class','country_of_origin','number_of_doors','safety_assessment','rating_name',
                'battery_capacity_KW_per_h','electric_range_km','charging_time_h']

    CD_ms_df = pd.DataFrame(columns = all_cols)
    sj=0
    for s, CD_model_s in CD_mt_df.iterrows():
        print(CD_model_s['ModelTypeLink'])
        if(CD_model_s['make'] == 'BMW'):

            ms_soup = beautiful_soup(CD_model_s['ModelTypeLink'])
            
            # Intially duting scraping this class was 'content_text' but now it has changed. 
            # Website keeps changing which causes issue during scraping
            # Security has been increased by adding cloudfare bot protection
            ms_results = ms_soup.find_all("table", class_='content_text') 
            if(len(ms_results) == 0):
                bypass_captcha(CD_model_s['ModelTypeLink'])
            else:
                ms_results = ms_soup.find_all("table", class_='table_versions')
                CD_ms_df, sj = scrapper(ms_results, CD_model_s, sj)
                

print('Scrapping completed!')

Scrapping completed!


/var/folders/zm/4w49slv95vd3qmv0snmz82tc0000gn/T/ipykernel_4111/1830828574.py:2: DtypeWarning: Columns (13,15,16,19,24,27,29,31,33,43,44,49,51,58,69,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  CD_ms_df = pd.read_csv ('../datasets/data.csv')


In [16]:
#CD_ms_df = CD_ms_df.dropna()
display(CD_ms_df)

# save to CSV file after scrapping
if(scrapData == 1):
    CD_ms_df.to_csv('data.csv', encoding='utf-8', index=False)

,make_model,make,model,generation,year_from,year_to,series,trim,body_type,load_height_mm,...,number_of_doors,safety_assessment,rating_name,battery_capacity_KW_per_h,electric_range_km,charging_time_h,male,female,unknown,total
0,acura cl,acura,cl,1 generation,1996.0,2000.0,coupe,2.2 at,coupe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0,10.0
1,acura cl,acura,cl,1 generation,1996.0,2000.0,coupe,2.2 mt,coupe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0,10.0
2,acura cl,acura,cl,1 generation,1996.0,2000.0,coupe,2.3 at,coupe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0,10.0
3,acura cl,acura,cl,1 generation,1996.0,2000.0,coupe,2.3 mt,coupe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0,10.0
4,acura cl,acura,cl,1 generation,1996.0,2000.0,coupe,3.0 at,coupe,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,0.0,0.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43042,zastava yugo koral,zastava,yugo koral,1 generation,1990.0,2008.0,hatchback,1.1i mt,hatchback,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68911.0,59474.0,3052.0,131437.0
43043,zastava yugo koral,zastava,yugo koral,1 generation,1990.0,2008.0,hatchback,1.3 mt,hatchback,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68911.0,59474.0,3052.0,131437.0
43044,zastava yugo koral,zastava,yugo koral,1 generation,1990.0,2008.0,hatchback,1.3i mt,hatchback,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68911.0,59474.0,3052.0,131437.0
43045,zastava yugo skala,zastava,yugo skala,1 generation,1990.0,2008.0,hatchback,1.1 mt,hatchback,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,68911.0,59474.0,3052.0,131437.0
